In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
import spotipy as sp

#import spotify keys
from config import client_id
from config import client_secret

In [2]:
#generate access token for spotify api
#from: https://stmorse.github.io/journal/spotify-api.html

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [3]:
# print(access_token)

In [4]:
#from: https://stmorse.github.io/journal/spotify-api.html

#base URL for Spotify API search
search_url = 'https://api.spotify.com/v1/search'

#In order to access, we send a properly formed GET request to the API server, with our access_token in the header. Let’s save this header info now, in the following very specific format:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [5]:
search_track = input("Please name a song: ")

Please name a song:  truth hurts


In [6]:
search_artist = input("Please name that song's artist: ")

Please name that song's artist:  lizzo


In [7]:
response = requests.get(search_url + '?q=track%3A' + search_track + '%20artist%3A' + search_artist + '&type=track%2Cartist&limit=1', headers=headers).json()

In [8]:
pprint(response)

{'artists': {'href': 'https://api.spotify.com/v1/search?query=track%3Atruth+hurts+artist%3Alizzo&type=artist&offset=0&limit=1',
             'items': [],
             'limit': 1,
             'next': None,
             'offset': 0,
             'previous': None,
             'total': 0},
 'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3Atruth+hurts+artist%3Alizzo&type=track&offset=0&limit=1',
            'items': [{'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/56oDRnqbIiwx4mymNEv7dS'},
                                              'href': 'https://api.spotify.com/v1/artists/56oDRnqbIiwx4mymNEv7dS',
                                              'id': '56oDRnqbIiwx4mymNEv7dS',
                                              'name': 'Lizzo',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:56oDRn

In [9]:
pprint(response['tracks']['items'][0]["id"])

'3HWzoMvoF3TQfYg4UPszDq'


In [10]:
track_id = response['tracks']['items'][0]["id"]

In [11]:
#base URL for Spotify API
base_url = 'https://api.spotify.com/v1/'

track_response = requests.get(base_url + 'audio-features/' + track_id, headers=headers).json()

In [12]:
#empty lists to store song data
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
id_num = []
track_href = []
analysis_url = []
duration_mins = []
time_signature = []

In [13]:
#fill in track info for each audio feature / empty list
danceability.append(track_response["danceability"])
energy.append(track_response["energy"])
key.append(track_response["key"])
loudness.append(track_response["loudness"])
mode.append(track_response["mode"])
speechiness.append(track_response["speechiness"])
acousticness.append(track_response["acousticness"])
instrumentalness.append(track_response["instrumentalness"])
liveness.append(track_response["liveness"])
valence.append(track_response["valence"])
tempo.append(track_response["tempo"])
id_num.append(track_response["id"])
track_href.append(track_response["track_href"])
analysis_url.append(track_response["analysis_url"])    
#duration recorded in api in milliseconds, converting to minutes here
duration_mins.append(round((track_response["duration_ms"] / 60000),2))
time_signature.append(track_response["time_signature"])    

In [14]:
#create a dictionary to hold data gathered from the api
track_dict = {
    "track_id": id_num,
    "danceability": danceability,
    "energy": energy,
    "key": key,
    "loudness": loudness,
    "mode": mode,
    "speechiness": speechiness,
    "acousticness": acousticness,
    "instrumentalness": instrumentalness,
    "liveness": liveness,
    "valence": valence,
    "tempo": tempo,
    "duration_mins": duration_mins,
    "time_signature": time_signature,
    "track_href": track_href,
    "analysis_url": analysis_url
    
}
#convert dictionary to a dataframe
input_track_df = pd.DataFrame(track_dict)
input_track_df.head()

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature,track_href,analysis_url
0,3HWzoMvoF3TQfYg4UPszDq,0.715,0.624,4,-3.046,0,0.114,0.11,0,0.123,0.412,158.087,2.89,4,https://api.spotify.com/v1/tracks/3HWzoMvoF3TQ...,https://api.spotify.com/v1/audio-analysis/3HWz...


In [15]:
#import global top songs csv and save as a df
global2022_df = pd.read_csv("resources/regional-global-weekly-2022-10-06.csv")
global2022_df.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams
0,1,spotify:track:3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),EMI,1,1,2,53478156
1,2,spotify:track:4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),Parlophone UK,2,2,6,33516367
2,3,spotify:track:4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,Columbia,1,3,27,32910077
3,4,spotify:track:2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",DALE PLAY Records,1,4,13,31691825
4,5,spotify:track:5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Sony Music Latin/La Industria,5,5,14,29821181


In [16]:
#extracting track id (removing "spotify:track:" from track id in csv)
track_id_series = global2022_df.uri.str[14:]

#convert to df and rename column
track_id_df = track_id_series.to_frame().rename(columns={"uri":"track_id"})

#merge dfs
global2022_df = global2022_df.merge(track_id_df, how="inner", left_index=True, right_index=True)
global2022_df = global2022_df[["track_id", "artist_names", "track_name"]]
global2022_df = global2022_df.astype({'track_id':'string','artist_names':'string','track_name':'string'})

In [17]:
global2022_df.head()

,track_id,artist_names,track_name
0,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras)
1,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue)
2,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was
3,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52"
4,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata


In [18]:
#empty lists to store song data
global_danceability = []
global_energy = []
global_key = []
global_loudness = []
global_mode = []
global_speechiness = []
global_acousticness = []
global_instrumentalness = []
global_liveness = []
global_valence = []
global_tempo = []
global_id_num = []
global_track_href = []
global_analysis_url = []
global_duration_mins = []
global_time_signature = []

#loop through dataframe, use track id to look up songs in spotify api and grab each one's audio features
for t in global2022_df["track_id"]:
    global_response = requests.get(base_url + 'audio-features/' + t, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    global_danceability.append(global_response["danceability"])
    global_energy.append(global_response["energy"])
    global_key.append(global_response["key"])
    global_loudness.append(global_response["loudness"])
    global_mode.append(global_response["mode"])
    global_speechiness.append(global_response["speechiness"])
    global_acousticness.append(global_response["acousticness"])
    global_instrumentalness.append(global_response["instrumentalness"])
    global_liveness.append(global_response["liveness"])
    global_valence.append(global_response["valence"])
    global_tempo.append(global_response["tempo"])
    global_id_num.append(global_response["id"])
    global_track_href.append(global_response["track_href"])
    global_analysis_url.append(global_response["analysis_url"])    
    #duration recorded in api in milliseconds, converting to minutes here
    global_duration_mins.append(round((global_response["duration_ms"] / 60000),2))
    global_time_signature.append(global_response["time_signature"])   
    
#create a dictionary to hold data gathered from the api
global_songs_dict = {
    "track_id": global_id_num,
    "danceability": global_danceability,
    "energy": global_energy,
    "key": global_key,
    "loudness": global_loudness,
    "mode": global_mode,
    "speechiness": global_speechiness,
    "acousticness": global_acousticness,
    "instrumentalness": global_instrumentalness,
    "liveness": global_liveness,
    "valence": global_valence,
    "tempo": global_tempo,
    "duration_mins": global_duration_mins,
    "time_signature": global_time_signature,
    "track_href": global_track_href,
    "analysis_url": global_analysis_url
    
}
#convert dictionary to a dataframe
gloabl_info_df = pd.DataFrame(global_songs_dict)
gloabl_info_df.head()

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature,track_href,analysis_url
0,3nqQXoyQOWXiESFLlDF1hG,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.266,0.238,131.121,2.62,4,https://api.spotify.com/v1/tracks/3nqQXoyQOWXi...,https://api.spotify.com/v1/audio-analysis/3nqQ...
1,4uUG5RXrOk84mYEfFvj3cK,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.371,0.304,128.040,2.92,4,https://api.spotify.com/v1/tracks/4uUG5RXrOk84...,https://api.spotify.com/v1/audio-analysis/4uUG...
2,4Dvkj6JhhA12EX05fT7y2e,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.311,0.662,173.930,2.79,4,https://api.spotify.com/v1/tracks/4Dvkj6JhhA12...,https://api.spotify.com/v1/audio-analysis/4Dvk...
3,2tTmW7RDtMQtBk7m2rYeSw,0.621,0.782,2,-5.548,1,0.0440,0.01250,0.033000,0.230,0.550,128.033,3.32,4,https://api.spotify.com/v1/tracks/2tTmW7RDtMQt...,https://api.spotify.com/v1/audio-analysis/2tTm...
4,5ww2BF9slyYgNOk37BlC4u,0.835,0.679,7,-5.329,0,0.0364,0.58300,0.000002,0.218,0.850,124.980,2.71,4,https://api.spotify.com/v1/tracks/5ww2BF9slyYg...,https://api.spotify.com/v1/audio-analysis/5ww2...


In [19]:
global_df = global2022_df.merge(gloabl_info_df, how="inner")
global_df

,track_id,artist_names,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature,track_href,analysis_url
0,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,131.121,2.62,4,https://api.spotify.com/v1/tracks/3nqQXoyQOWXi...,https://api.spotify.com/v1/audio-analysis/3nqQ...
1,4uUG5RXrOk84mYEfFvj3cK,"David Guetta, Bebe Rexha",I'm Good (Blue),0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,2.92,4,https://api.spotify.com/v1/tracks/4uUG5RXrOk84...,https://api.spotify.com/v1/audio-analysis/4uUG...
2,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,0.520,0.731,6,-5.338,0,0.0557,0.34200,0.001010,0.3110,0.662,173.930,2.79,4,https://api.spotify.com/v1/tracks/4Dvkj6JhhA12...,https://api.spotify.com/v1/audio-analysis/4Dvk...
3,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",0.621,0.782,2,-5.548,1,0.0440,0.01250,0.033000,0.2300,0.550,128.033,3.32,4,https://api.spotify.com/v1/tracks/2tTmW7RDtMQt...,https://api.spotify.com/v1/audio-analysis/2tTm...
4,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,0.835,0.679,7,-5.329,0,0.0364,0.58300,0.000002,0.2180,0.850,124.980,2.71,4,https://api.spotify.com/v1/tracks/5ww2BF9slyYg...,https://api.spotify.com/v1/audio-analysis/5ww2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,4woTEX1wYOTGDqNXuavlRC,"Eminem, Dina Rae",Superman,0.802,0.755,4,-3.138,0,0.0564,0.02150,0.000000,0.2950,0.580,130.027,5.84,4,https://api.spotify.com/v1/tracks/4woTEX1wYOTG...,https://api.spotify.com/v1/audio-analysis/4woT...
196,003vvx7Niy0yvhvHt4a68B,The Killers,Mr. Brightside,0.352,0.911,1,-5.230,1,0.0747,0.00121,0.000000,0.0995,0.236,148.033,3.72,4,https://api.spotify.com/v1/tracks/003vvx7Niy0y...,https://api.spotify.com/v1/audio-analysis/003v...
197,2K7xn816oNHJZ0aVqdQsha,The Neighbourhood,Softcore,0.575,0.568,9,-5.509,0,0.0300,0.04840,0.000417,0.2860,0.370,93.986,3.44,4,https://api.spotify.com/v1/tracks/2K7xn816oNHJ...,https://api.spotify.com/v1/audio-analysis/2K7x...
198,4gzsuuZypVbxs0Af1LSZyB,"Tainy, Bad Bunny, Julieta Venegas",Lo Siento BB:/ (with Bad Bunny & Julieta Venegas),0.639,0.703,4,-6.330,0,0.0794,0.08890,0.000002,0.0865,0.138,169.888,3.46,4,https://api.spotify.com/v1/tracks/4gzsuuZypVbx...,https://api.spotify.com/v1/audio-analysis/4gzs...
